In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# 이벤트 시간(current_time) < 응급실 도착 시간(ed_arrival) 문제 해결  
- 해결 전략: 해당 이벤트의 시간을 아래 우선순위에 따라 이벤트의 1분 전으로 이동   
(단, 응급실 도착 시간보다는 같거나 커야 함)  
1. 1순위 (진단 단계): ECG 또는 Troponin 관련 이벤트가 있다면?  
-> 그 중 가장 빠른 이벤트의 1분 전으로 이동  
(해석: "본격적인 검사 직전(또는 동시)에 이루어진 초기 처치/오더로 간주")  
2. 2순위 (치료/종료 단계): 1순위가 없고 PCI, ICU, Discharge, Death가 있다면?  
-> 그 중 가장 빠른 이벤트의 1분 전으로 이동   
(해석: "주요 처치나 퇴원 전에 이루어진 기록으로 간주")  
3. 3순위 (기타): 위 앵커들이 모두 없다면?  
-> 응급실 도착 시간 + 1분으로 이동


- ID 기준 로직 적용: 문자열 대신 event_id == 1 (ED_ARRIVAL)을 기준으로 타임스탬프 보정  
- 강제 정렬: 시간이 같을 경우 무조건 ID 1이 맨 위로 오도록 정렬 우선순위를 부여  

# 동적 피처 계산
- 계산 항목: prefix_len, time_since_ed, time_since_last, is_night, cum_ecg_cnt, cum_trop_cnt, stemi_flag, trop_pos_flag, current_event_id, time_since_start_min, pci_status, last_trop, run_max_trop, trop_trend  
- 트로포닌 관련 변수(last_trop 등)는 데이터에 검사 수치가 없다면 0으로 처리

# 수행 작업
- ED_ARRIVAL보다 시간이 빠른 이상치를 찾아 임상적 우선순위(진단/치료)에 따라 재배치  
- 정렬: ED_ARRIVAL이 무조건 시퀀스 맨 앞에 오도록 강제 정렬  
- 계산: 모든 동적 변수(14종) 및 로그 변수(3종) 계산

In [ ]:
import pandas as pd
import numpy as np
import os

# =========================================================
# 1. 설정 및 데이터 로드
# =========================================================
base_path = '/content/drive/MyDrive/Colab Notebooks/2025-2 데이터애널리틱스'
input_file = 'cohort_ver150_event_level_full.csv'
path_in = os.path.join(base_path, input_file)

print(f"📂 데이터 로드 중... ({input_file})")
if not os.path.exists(path_in):
    raise FileNotFoundError(f"❌ 파일을 찾을 수 없습니다: {path_in}")

df = pd.read_csv(path_in)

# ---------------------------------------------------------
# 1-1. 컬럼명 표준화 (timestamp)
# ---------------------------------------------------------
if 'timestamp' not in df.columns:
    for col in ['charttime', 'event_time', 'intime', 'starttime']:
        if col in df.columns:
            print(f"   ℹ️ 시간 컬럼 변경: {col} -> timestamp")
            df = df.rename(columns={col: 'timestamp'})
            break
df['timestamp'] = pd.to_datetime(df['timestamp'])

# ---------------------------------------------------------
# 1-2. Event ID 처리
# ---------------------------------------------------------
if 'event_name' not in df.columns:
    for col in ['current_event', 'event', 'type', 'event_id']:
        if col in df.columns:
            df = df.rename(columns={col: 'event_name'})
            break

# event_name이 숫자인지 문자인지 확인하여 event_id 생성
sample_val = df['event_name'].iloc[0]
is_numeric = isinstance(sample_val, (int, float, np.number)) or str(sample_val).isdigit()

if is_numeric:
    print("ℹ️ event_name이 이미 숫자(ID)입니다. 이를 event_id로 사용합니다.")
    df['event_id'] = df['event_name'].astype(int)
else:
    print("🔄 문자열 이벤트를 ID로 매핑합니다.")
    EVENT_MAP = {
        'ED_ARRIVAL': 1, 'ED_DEPARTURE': 2, 'ECG_STEMI_FLAG': 3, 'ECG_TAKEN': 4,
        'TROP_POSITIVE': 5, 'TROP_TAKEN': 6, 'ANTI_PLT_ADMIN': 7, 'ANTI_PLT_ORDER': 8,
        'PCI_START': 9, 'DEATH': 10, 'DISCHARGE': 11, 'ICU_INTIME': 12, 'ICU_OUTTIME': 13, 'EOS': 14
    }
    df['event_id'] = df['event_name'].map(EVENT_MAP).fillna(0).astype(int)

# =========================================================
# 2. 타임스탬프 보정 (우선순위 로직 + 시작점 강제)
# =========================================================
print("🔧 타임스탬프 보정 적용 중...")

ANCHOR_DIAG = [2, 3, 4, 5]  # STEMI, ECG, Trop+, Trop
ANCHOR_TREAT = [8, 12, 11, 10, 9] # PCI, ICU, Disch, Death, ED_Dep

def correct_timestamp_logic(group):
    # 1. ED_ARRIVAL 시간 확인
    arrival_rows = group[group['event_id'] == 1]
    if arrival_rows.empty:
        arrival_time = group['timestamp'].min()
    else:
        arrival_time = arrival_rows['timestamp'].min()

    # 2. 도착보다 빠른 이벤트 찾기
    mask_early = group['timestamp'] < arrival_time
    if not mask_early.any():
        return group

    # 3. 앵커 탐색
    anchor_1 = group[(group['event_id'].isin(ANCHOR_DIAG)) & (group['timestamp'] >= arrival_time)]['timestamp'].min()
    anchor_2 = group[(group['event_id'].isin(ANCHOR_TREAT)) & (group['timestamp'] >= arrival_time)]['timestamp'].min()

    # 4. 타겟 시간 결정
    if pd.notna(anchor_1):
        target_time = anchor_1 - pd.Timedelta(minutes=1)
    elif pd.notna(anchor_2):
        target_time = anchor_2 - pd.Timedelta(minutes=1)
    else:
        target_time = arrival_time + pd.Timedelta(minutes=1)

    if target_time < arrival_time: target_time = arrival_time

    # 5. 업데이트
    group.loc[mask_early, 'timestamp'] = target_time
    return group

df = df.groupby('hadm_id', group_keys=False).apply(correct_timestamp_logic)

# 정렬 우선순위: ID 1(ED_ARRIVAL)이 항상 맨 앞
df['sort_priority'] = np.where(df['event_id'] == 1, 0, 1)
df = df.sort_values(by=['hadm_id', 'timestamp', 'sort_priority', 'event_id'])

# =========================================================
# 3. 동적 변수 계산 (14종 + Log)
# =========================================================
print("🔄 동적 변수 계산 중...")

# (1) 시퀀스/시간 변수
df['prefix_len'] = df.groupby('hadm_id').cumcount() + 1

first_times = df.groupby('hadm_id')['timestamp'].transform('first')
df['time_since_ed'] = (df['timestamp'] - first_times).dt.total_seconds() / 60.0
df['time_since_ed'] = df['time_since_ed'].clip(lower=0)

df['prev_time'] = df.groupby('hadm_id')['timestamp'].shift(1)
df['time_since_last'] = (df['timestamp'] - df['prev_time']).dt.total_seconds() / 60.0
df['time_since_last'] = df['time_since_last'].fillna(0).clip(lower=0)

df['time_since_start_min'] = df['time_since_ed']

# 로그 변환
df['log_time_since_ed'] = np.log1p(df['time_since_ed'])
df['log_time_since_last'] = np.log1p(df['time_since_last'])
df['log_time_since_start_min'] = np.log1p(df['time_since_start_min'])

# (2) 상태/누적 변수
df['is_night'] = ((df['timestamp'].dt.hour >= 22) | (df['timestamp'].dt.hour < 7)).astype(int)
df['cum_ecg_cnt'] = df.groupby('hadm_id')['event_id'].apply(lambda x: (x == 3).cumsum()).values
df['cum_trop_cnt'] = df.groupby('hadm_id')['event_id'].apply(lambda x: (x == 5).cumsum()).values
df['stemi_flag'] = df.groupby('hadm_id')['event_id'].apply(lambda x: (x == 2).cummax().astype(int)).values
df['trop_pos_flag'] = df.groupby('hadm_id')['event_id'].apply(lambda x: (x == 4).cummax().astype(int)).values
df['pci_status'] = df.groupby('hadm_id')['event_id'].apply(lambda x: (x == 8).cummax().astype(int)).values

# (3) Troponin 수치
trop_val_col = 'valuenum' if 'valuenum' in df.columns else ('value' if 'value' in df.columns else None)
if trop_val_col:
    df['temp_val'] = pd.to_numeric(df[trop_val_col], errors='coerce')
    is_trop = df['event_id'].isin([4, 5])
    df['temp_val'] = np.where(is_trop, df['temp_val'], np.nan)

    df['last_trop'] = df.groupby('hadm_id')['temp_val'].ffill().fillna(0)
    df['run_max_trop'] = df.groupby('hadm_id')['temp_val'].cummax().ffill().fillna(0)
    prev_trop = df.groupby('hadm_id')['temp_val'].shift(1)
    df['trop_trend'] = np.where(is_trop, df['temp_val'] - prev_trop, 0)
    df['trop_trend'] = df['trop_trend'].fillna(0)
    df = df.drop(columns=['temp_val'])
else:
    df['last_trop'] = 0.0; df['run_max_trop'] = 0.0; df['trop_trend'] = 0.0

df['current_event_id'] = df['event_id']
df = df.drop(columns=['sort_priority', 'prev_time'], errors='ignore')

print("✅ [Cell 1] 동적 변수 계산 완료.")

📂 데이터 로드 중... (cohort_ver150_event_level_full.csv)
ℹ️ event_name이 이미 숫자(ID)입니다. 이를 event_id로 사용합니다.
🔧 타임스탬프 보정 적용 중...


/tmp/ipython-input-1891275610.py:93: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df = df.groupby('hadm_id', group_keys=False).apply(correct_timestamp_logic)


🔄 동적 변수 계산 중...
✅ [Cell 1] 동적 변수 계산 완료.


# 타겟 변수 계산  
1. 동시에 발생한 이벤트의 순서 (Tie-breaking)상황: 10:00에 ECG_TAKEN(ID 3)과 TROP_TAKEN(ID 5)가 동시에 기록된 경우  
-> event_id 오름차순으로 정렬  
2. 마지막 이벤트의 타겟 (End of Sequence)상황: 환자의 기록이 DISCHARGE(퇴원)로 끝난 경우  
-> target_next_evt: 14 (EOS) 로 설정 & target_time_to_next: 0분으로 설정 (퇴원 후 '다음 사건 없음(EOS)'까지 걸리는 시간을 0으로 봄)

In [ ]:
# =========================================================
# 4. 타겟 변수 계산 (Target 1, 2, 3)
# =========================================================
print("🎯 타겟 변수 3종 계산 중 (Target 3: Remaining LOS)...")

# Target 1: Next Event ID (Classification)
# 마지막 이벤트의 다음은 EOS(14)로 설정
df['target_next_evt'] = df.groupby('hadm_id')['event_id'].shift(-1).fillna(14).astype(int)

# Target 2: Time to Next Event (Regression)
# 마지막 이벤트의 남은 시간은 0으로 설정
next_times = df.groupby('hadm_id')['time_since_ed'].shift(-1)
df['target_time_to_next'] = (next_times - df['time_since_ed']).fillna(0).clip(lower=0)

# Target 3: Remaining LOS (Regression)
# [수정된 로직] 퇴원(마지막 이벤트)까지 남은 시간 = (마지막 시간 - 현재 시간)
# 각 환자의 마지막 이벤트 시간을 '퇴원 시간(dischtime)'으로 간주
disch_times = df.groupby('hadm_id')['timestamp'].transform('max')
df['target_remain_los'] = (disch_times - df['timestamp']).dt.total_seconds() / 60.0
# 음수 방지 및 로그 변환 추가 (학습용)
df['target_remain_los'] = df['target_remain_los'].clip(lower=0)
df['log_target_remain_los'] = np.log1p(df['target_remain_los'])

# =========================================================
# 5. 저장 및 검증
# =========================================================
output_file = 'cohort_ver128_final_targets_added.csv'
path_out = os.path.join(base_path, output_file)

# 컬럼 순서 및 선택
cols_to_keep = [
    'hadm_id', 'prefix_len', 'event_id', 'event_name', 'timestamp',
    'time_since_ed', 'time_since_last', 'time_since_start_min',
    'log_time_since_ed', 'log_time_since_last', 'log_time_since_start_min',
    'is_night', 'cum_ecg_cnt', 'cum_trop_cnt', 'stemi_flag', 'trop_pos_flag',
    'pci_status', 'last_trop', 'run_max_trop', 'trop_trend',
    'current_event_id',
    'target_next_evt',        # Target 1
    'target_time_to_next',    # Target 2
    'target_remain_los',      # Target 3 (New)
    'log_target_remain_los'   # Target 3 Log (Optional/Recommended)
]
final_cols = [c for c in cols_to_keep if c in df.columns]
df_final = df[final_cols].copy()

print("\n✅ 최종 데이터 검증 (상위 5행)")
print(df_final[['hadm_id', 'prefix_len', 'event_id', 'time_since_ed', 'target_remain_los']].head().to_markdown(index=False))

df_final.to_csv(path_out, index=False)
print(f"\n🎉 저장 완료: {output_file}")
print("   - Target 3: 'target_remain_los' (퇴원까지 남은 시간) 적용 완료")

🎯 타겟 변수 3종 계산 중 (Target 3: Remaining LOS)...

✅ 최종 데이터 검증 (상위 5행)
|     hadm_id |   prefix_len |   event_id |   time_since_ed |   target_remain_los |
|------------:|-------------:|-----------:|----------------:|--------------------:|
| 2.00063e+07 |            1 |          1 |               0 |               17408 |
| 2.00063e+07 |            2 |          3 |             160 |               17248 |
| 2.00063e+07 |            3 |          6 |             160 |               17248 |
| 2.00063e+07 |            4 |          7 |             160 |               17248 |
| 2.00063e+07 |            5 |          2 |             161 |               17247 |

🎉 저장 완료: cohort_ver128_final_targets_added.csv
   - Target 3: 'target_remain_los' (퇴원까지 남은 시간) 적용 완료


In [40]:
import pandas as pd
import numpy as np
import os

# =========================================================
# 1. 파일 로드 (직전 단계 결과물)
# =========================================================
base_path = '/content/drive/MyDrive/Colab Notebooks/2025-2 데이터애널리틱스'
input_file = 'cohort_ver128_final_targets_added.csv'
path_in = os.path.join(base_path, input_file)

print(f"📂 데이터 로드 중... ({input_file})")
if not os.path.exists(path_in):
    raise FileNotFoundError(f"❌ 파일을 찾을 수 없습니다: {path_in}")

df = pd.read_csv(path_in)

# =========================================================
# 2. target_mortality 계산
# =========================================================
print("🎯 Target 4: target_mortality (사망 여부) 계산 중...")

# 로직: 해당 환자(hadm_id)의 이벤트 목록 중 'DEATH (ID=10)'가 하나라도 있으면 1, 아니면 0
# transform('max')를 사용하여 환자별로 동일한 값(Static Target)을 부여합니다.
df['target_mortality'] = df.groupby('hadm_id')['event_id'].transform(lambda x: (x == 10).max()).astype(int)

# =========================================================
# 3. 저장 및 확인
# =========================================================
output_file = 'cohort_ver128_final_all_targets.csv' # 최종_진짜_최종 파일명
path_out = os.path.join(base_path, output_file)

# 컬럼 순서 정리 (타겟 변수 4개 확인)
cols_to_keep = [
    'hadm_id', 'prefix_len', 'event_id', 'event_name', 'timestamp',
    'time_since_ed', 'time_since_last', 'time_since_start_min',
    'log_time_since_ed', 'log_time_since_last', 'log_time_since_start_min',
    'is_night', 'cum_ecg_cnt', 'cum_trop_cnt', 'stemi_flag', 'trop_pos_flag',
    'pci_status', 'last_trop', 'run_max_trop', 'trop_trend',
    'current_event_id',
    'target_next_evt',        # Target 1 (Classification)
    'target_time_to_next',    # Target 2 (Regression)
    'target_remain_los',      # Target 3 (Regression)
    'log_target_remain_los',  # Target 3 Log
    'target_mortality'        # Target 4 (Binary Classification)
]
# 실제 있는 컬럼만 선택
final_cols = [c for c in cols_to_keep if c in df.columns]
df_final = df[final_cols].copy()

print("\n✅ 최종 데이터 검증 (사망 환자 샘플)")
# 사망한 환자 1명만 뽑아서 확인
death_hadm = df_final[df_final['target_mortality'] == 1]['hadm_id'].iloc[0]
print(df_final[df_final['hadm_id'] == death_hadm][['hadm_id', 'event_name', 'target_mortality']].tail().to_markdown(index=False))

df_final.to_csv(path_out, index=False)
print(f"\n🎉 모든 타겟 추가 완료! 저장된 파일: {output_file}")

📂 데이터 로드 중... (cohort_ver128_final_targets_added.csv)
🎯 Target 4: target_mortality (사망 여부) 계산 중...

✅ 최종 데이터 검증 (사망 환자 샘플)
|     hadm_id |   event_name |   target_mortality |
|------------:|-------------:|-------------------:|
| 2.01495e+07 |            5 |                  1 |
| 2.01495e+07 |            5 |                  1 |
| 2.01495e+07 |            5 |                  1 |
| 2.01495e+07 |            3 |                  1 |
| 2.01495e+07 |           10 |                  1 |

🎉 모든 타겟 추가 완료! 저장된 파일: cohort_ver128_final_all_targets.csv


In [41]:
print(df_final)

        hadm_id  prefix_len  event_id  event_name            timestamp  \
0      20006266           1         1           1  2179-04-27 10:54:00   
1      20006266           2         3           3  2179-04-27 13:34:00   
2      20006266           3         6           6  2179-04-27 13:34:00   
3      20006266           4         7           7  2179-04-27 13:34:00   
4      20006266           5         2           2  2179-04-27 13:35:00   
...         ...         ...       ...         ...                  ...   
40812  29997425          68         7           7  2163-11-26 08:00:00   
40813  29997425          69         3           3  2163-11-26 08:56:00   
40814  29997425          70         3           3  2163-11-29 11:50:00   
40815  29997425          71         3           3  2163-11-29 17:29:00   
40816  29997425          72        11          11  2163-11-30 14:30:00   

       time_since_ed  time_since_last  time_since_start_min  \
0                0.0              0.0           

In [42]:
print(df_final.columns)

Index(['hadm_id', 'prefix_len', 'event_id', 'event_name', 'timestamp',
       'time_since_ed', 'time_since_last', 'time_since_start_min',
       'log_time_since_ed', 'log_time_since_last', 'log_time_since_start_min',
       'is_night', 'cum_ecg_cnt', 'cum_trop_cnt', 'stemi_flag',
       'trop_pos_flag', 'pci_status', 'last_trop', 'run_max_trop',
       'trop_trend', 'current_event_id', 'target_next_evt',
       'target_time_to_next', 'target_remain_los', 'log_target_remain_los',
       'target_mortality'],
      dtype='object')
